In [1]:
import numpy as np
import os, glob
# import gdal, osr # python2.7 - installed in?
from osgeo import gdal
import matplotlib
from matplotlib import pyplot as plt
import PIL
from PIL import Image
from datetime import datetime
import matplotlib.cm as cm
import matplotlib as mpl
import os
import rasterio


In [2]:
now = datetime.now()
print(now)

2021-12-14 18:10:55.826152


In [3]:
matplotlib.__version__

'3.3.4'

In [4]:
gdal.VersionInfo()

'2040200'

## set the date to label the plot

In [5]:
plot_date = "test_25_april_2016"
print(plot_date)


saved_img_name = "test_25_april_2016.jpg"

print('saved img label will be: %s' % saved_img_name)

test_25_april_2016
saved img label will be: test_25_april_2016.jpg


In [6]:
save_plot_dir = "/media/ehsan/6T_part1/14528_apr2016/toa_refl_april_2016_9cam4bands_day1_30_p1_233_b1_46/toa_files_in_range_2016_4_25/rasters_noDataNeg99_TiffFileFloat64_max"
print(os.path.isdir(save_plot_dir))
print(save_plot_dir)

True
/media/ehsan/6T_part1/14528_apr2016/toa_refl_april_2016_9cam4bands_day1_30_p1_233_b1_46/toa_files_in_range_2016_4_25/rasters_noDataNeg99_TiffFileFloat64_max


## set directory path to mosaic file 

In [7]:
#~ float dtype
# raster_fullpath = '/Users/ehsanmos/Documents/RnD/MISR_lab/mosaic_and_rasters.nosync/mosaic_fromVRT_byte_allDays_test_p75_180.tif'

raster_dir = "/media/ehsan/6T_part1/14528_apr2016/toa_refl_april_2016_9cam4bands_day1_30_p1_233_b1_46/toa_files_in_range_2016_4_25/rasters_noDataNeg99_TiffFileFloat64_max/"

raster_name = "mosaic_fromVRT_float64_nearest_25-april-2016.tif"
# raster_name = "raster_path_164_block_20_85GCPs_noGdalRes_dTypeFloat64_85gcps_latlon_reprojToEPSG_3995.tif"

raster_fullpath = os.path.join(raster_dir, raster_name)

print(raster_fullpath)


/media/ehsan/6T_part1/14528_apr2016/toa_refl_april_2016_9cam4bands_day1_30_p1_233_b1_46/toa_files_in_range_2016_4_25/rasters_noDataNeg99_TiffFileFloat64_max/mosaic_fromVRT_float64_nearest_25-april-2016.tif


In [8]:
#~ check if ratster exist
if (os.path.isfile(raster_fullpath)):
    print('raster found!')
else:
    print('raster NOT exist!')
    raise SystemExit()

raster found!


In [9]:
## read data from raster in to array
input_dataset = gdal.Open(raster_fullpath)
print('dataset type: %s' %type(input_dataset))


dataset type: <class 'osgeo.gdal.Dataset'>


In [10]:
# raster_band = input_dataset.GetRasterBand(1)

In [11]:
# type(raster_band)

## build a dictionary to fill w/ metadata

In [12]:
metadata = {}
print(type(metadata))

<class 'dict'>


## we create metadata

In [13]:
metadata['array_rows'] = input_dataset.RasterYSize
metadata['array_cols'] = input_dataset.RasterXSize

print('rows: %s' % metadata['array_rows'])
print('columns: %s' % metadata['array_cols'])

rows: 34487
columns: 34627


## geoTransform info

In [14]:
## extract geotransforn from raster

geotransform = input_dataset.GetGeoTransform()
print(geotransform) # note: units in meters
print('order of geoTransform matrix: topleftX, pixelW, 0, topleftY, 0, pixelH')

(-3829892.185075982, 221.0208586449649, 0.0, 3813492.6174702984, 0.0, -221.0208586449649)
order of geoTransform matrix: topleftX, pixelW, 0, topleftY, 0, pixelH


In [15]:
## find corners

xMin = geotransform[0]
xMax = geotransform[0] + input_dataset.RasterXSize*geotransform[1]        # i changed it to * to get xMax value
yMin = geotransform[3] + input_dataset.RasterYSize*geotransform[5]
yMax = geotransform[3]

In [16]:
print(xMin)
print(xMax)
print(yMin)
print(yMax)

-3829892.185075982
3823397.0872232174
-3808853.7346186065
3813492.6174702984


In [17]:
# metadata['extent'] = (xMin,xMax,yMin,yMax)
plot_extent = (xMin,xMax,yMin,yMax)
# plot_extent = (-3200000,3200000,-3200000,3200000)
plot_extent

(-3829892.185075982,
 3823397.0872232174,
 -3808853.7346186065,
 3813492.6174702984)

#################################################################################################

## get shape of roughness data

In [18]:
# # array_shape = raster.ReadAsArray(0,0, metadata['array_cols'], metadata['array_rows']).astype('byte').shape # check this f(.)
# array_shape = raster_band.ReadAsArray(0,0, metadata['array_cols'], metadata['array_rows']).astype('float64').shape # check this f(.)

# print(array_shape)

In [19]:

# input_img = Image.open(raster_fullpath)
input_img = rasterio.open(raster_fullpath)
type(input_img)
rough_arr = input_img.read(1)

In [20]:
# input_img.transform

In [21]:
rough_arr.shape

(34487, 34627)

In [22]:
rough_arr.dtype

dtype('float64')

## build an array with zeros

In [23]:
# rough_arr = np.zeros((array_shape[0], array_shape[1], dataset.RasterCount), dtype='byte') # check f(.) arguments
# rough_arr = np.zeros((array_shape[0], array_shape[1], dataset.RasterCount), dtype='float64') # check f(.) arguments


In [24]:
# print(rough_arr.dtype)
# print(rough_arr.min())
# print(rough_arr.max())

## read roughess data inside array and fill zero array with it

In [25]:
# rough_arr = raster.ReadAsArray(0, 0, metadata['array_cols'], metadata['array_rows']).astype('byte')
# rough_arr = raster_band.ReadAsArray(0, 0, metadata['array_cols'], metadata['array_rows']).astype('float64')


In [26]:
# print(type(rough_arr))
print(rough_arr.dtype)

print('array shape: (%s, %s)' % rough_arr.shape)
print('roughness min: %s' % rough_arr.min())
print('roughness max: %s' % rough_arr.max())

float64
array shape: (34487, 34627)
roughness min: -99.0625
roughness max: 113.61127471923828


## >>> plot section ...

In [27]:
print('-> making plot...')

-> making plot...


## using PIL library to check visual of image

In [28]:
# #~~ create PIL image from roughness array
# rough_PIL_img = Image.fromarray(rough_arr)
# print(type(rough_PIL_img))
# rough_PIL_img.show()

## test if we can mask negative pixels

In [29]:
# # mask_cond = (rough_arr >= 0.0) & (rough_arr == -999994.0)  # elementwize and; returns boolean mask 
# # print(mask_cond)

# #~ test2 w/np.where 
# np.where((rough_arr >= 0.0) & (rough_arr == -999994.0), np.NAN, rough_arr)

# # rough_arr[mask_cond] = np.nan # nan rough values and land mask pixels 
# plt.imshow(rough_arr, cmap='gray') # plot the rest of data



## >>> build mask arrays (Q- how to mask array? and overlay/plot both over each other?)

In [30]:
#~ 1st, update -99 --> Nan 
# rough_arr[rough_arr == -99.0] = np.nan

## 1-filter for roughness

In [31]:
# land_mask = np.ma.masked_values(roughness_array, roughness_array == -999994.0)  # for float data
# land_mask = np.ma.masked_array(roughness_array==-999994.0, roughness_array)
# land_mask = np.ma.masked_where(roughness_array==-999994.0, roughness_array)
# land_mask = np.where(rough_arr == -999994.0, 0, 1.0)
# land_mask = np.ma.masked_where(roughness_array == -999994.0, roughness_array)
# land_mask = np.ma.masked_array(roughness_array == -999994, roughness_array)



#~ filter roughness array
#~ make a copy of roughness array to use it for filter
roughness = np.copy(rough_arr)

##
print(np.nanmin(roughness))
print(np.nanmax(roughness))

# mask neg. values and keep +values; to extract positive rough values
print("mask negative values")
roughness[rough_arr <= 0.0] = np.nan 

print('min roughness: %s' % np.nanmin(roughness))
print('max roughness: %s' % np.nanmax(roughness))

print(np.count_nonzero(roughness[roughness > 0]))
print(type(roughness))


-99.0625
113.61127471923828
mask negative values
min roughness: 1.1026859283447266e-06
max roughness: 113.61127471923828
369786771
<class 'numpy.ndarray'>


## 2-filter for land mask

In [32]:
# #~ filter for land mask
# # land_mask = rough_arr  # make copy of rough_arr; not good cuz if we modify rough_arr ==> land_mask will change
# land = np.copy(rough_arr)  # right way to copy array 

# print('min: %s' % np.nanmin(land))
# print('max: %s' % np.nanmax(land))

# print("mask land data")
# land[rough_arr != -999994.0] = np.nan  # keep -999994 and masks everything.

# print('landMask min: %s' % np.nanmin(land))
# print('landMask max: %s' % np.nanmax(land))

# print(np.count_nonzero(land[land == -999994.0]))
# # plt.imshow(land, cmap='gray')


## 3-filter for noData

In [33]:
# #~ filter for noData (rest of data)
# nodata = np.copy(rough_arr)  # right way to copy array 
# print('min: %s' % np.nanmin(nodata))
# print('max: %s' % np.nanmax(nodata))

# print("filter no-data")
# nodata[rough_arr != -99.0] = np.nan  # -99.0 is signal/flag for noData

# print('noData min: %s' % np.nanmin(nodata))
# print('noData max: %s' % np.nanmax(nodata))

# # plt.imshow(nodata, origin='lower', cmap='gray')

# #~ np.where with 2 conditions for noData???
# # nodata_mask = np.where((rough_arr >= 0.0) & (rough_arr==-999994.0), np.nan, -9.)
# # print(nodata_mask)

In [34]:
# other = np.copy(rough_arr)
# # other[other != -999994.0] = np.nan # mask everything eccept this
# # other[other != -1] = np.nan # mask everything eccept this
# # other[other <= 0 ] = np.nan # mask neg. values
# other[other != 0 ] = np.nan # mask everything eccept zero

# plt.imshow(other, cmap='Reds', origin='lower')

## customed colormap from default colormap

In [35]:
''' colormap is used to map data to colors == colorcoded data'''
from matplotlib.colors import ListedColormap

#~~ seaice roughness colormap
blues_for_seaice = cm.get_cmap('Blues', 100)
seaice_color = ListedColormap(blues_for_seaice(np.linspace(0.1, 1, 100)))
SIR_colormap = plt.cm.get_cmap(seaice_color, 10)
SIR_colormap.set_over('red')  # to set anything onder zero==black
# SIR_colormap.set_under('black')






# #~~ landcolormap
# land_cmap = cm.get_cmap('gray', 100)
# # print(land_cmap)
# land_color = ListedColormap(land_cmap(np.linspace(0.2, 0.25, 100)))  # should be in range: [0, 1]
# # print(land_color)  # this is the new sequential colormap and we will use the colorcode of our defined/extracted range from the full colormap

# #~~ noData colormap
# grays_for_nodata = cm.get_cmap('gray', 100)
# nodata_color = ListedColormap(grays_for_nodata(np.linspace(0.0, 0.01, 100)))

# #~ inspired by: https://matplotlib.org/3.2.1/tutorials/colors/colormap-manipulation.html

## plot arrays (Q- how to overlay 2 arrays/images?)

In [ ]:
(fig, ax) = plt.subplots()  # returns one figure==canvas and we can have multiple subplots==axes; has more features than plt.subplot(); figure != plot = image; e.g: fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(6, 5.4))

print(type(fig))
print(type(ax))

'''
origin='lower'
normalization=None == not linear scale data into [lowest=0,highest=1] & we pick the data range to plot [0,100]
'''
#~~ transpose data to solve flipping issue that imshow() caused: https://github.com/bokeh/bokeh/issues/1666
roughness=roughness.T
# land=land.T
# nodata=nodata.T

img1 = ax.imshow(roughness, cmap=SIR_colormap , origin='lower', interpolation='none', extent=plot_extent, norm=None, vmin=0.0, vmax=200);
# img2 = ax.imshow(land, cmap= land_color, origin='lower', interpolation='none', extent=plot_extent)
# img3 = ax.imshow(nodata, cmap=nodata_color, origin='lower', interpolation='none', extent=plot_extent);

#~ colorbar
cmap_title = 'Sea Ice Roughness (cm)'
cbar = fig.colorbar(img1, extend='max', aspect=40, fraction=0.012, pad=0.02);  # extend shows the range that goes beyond max limit on plot
cbar.set_label(cmap_title, rotation=90, labelpad=10, fontsize=5);
cbar.ax.tick_params(labelsize=5)
cbar.outline.set_visible(False)


# plt.title('Sea Ice Roughness over the Arctic'); 
ax.set_title('MISR-Estimated Sea Ice Roughness for ' + plot_date, fontsize=8);

#~ axis
# ax = plt.gca();  # get-current-axis
ax.ticklabel_format(useOffset=False, style='plain', size=10); # do not use scientific notation 
rotatexlabels = plt.setp(ax.get_xticklabels(), rotation=90); # rotate x tick labels 90 degrees

#~ axis limits
xstart, xend = ax.get_xlim()
xstepsize = abs(xend-xstart)/6
ax.xaxis.set_ticks(np.arange(xstart, xend, xstepsize))


ystart, yend = ax.get_ylim()
ystepsize = abs(yend-ystart)/6
ax.yaxis.set_ticks(np.arange(ystart, yend, ystepsize))

#~ axis font size
plt.xticks(fontsize=3)
plt.yticks(fontsize=3)

#~ grid
ax.grid(True, which='both', axis='both', color='white', linewidth=0.05)

#~ x and y labels
plt.xlabel('Longitude (m)', fontsize=5)
plt.ylabel('Latitude (m)', fontsize=5)

#~ path to save directory
saved_img_fullpath = os.path.join(save_plot_dir, saved_img_name)

print('-> START saving image on disc!')
plt.savefig(saved_img_fullpath, bbox_inches='tight', pad_inches=0.2, dpi=600)
print(saved_img_fullpath)
print('-> FINISHED saving image on disc!')

# plt.show()     # should come after plt.savefig()

plt.close()    # close the figure window; plt.close(fig) if we created by:  fig, ax = plt.subplots()


# inspired by: 
# https://www.neonscience.org/resources/learning-hub/tutorials/mask-raster-py
# https://www.neonscience.org/resources/learning-hub/tutorials/neon-hsi-aop-functions-tiles-py


<class 'matplotlib.figure.Figure'>
<class 'matplotlib.axes._subplots.AxesSubplot'>


In [ ]:
dataset = None